In [117]:
# 导入模块
import requests
import re

In [118]:
def get_responses(date):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    url = f'http://www.cninfo.com.cn/data/statis/getMarketStatisticsData?sdate={date}&edate={date}&platecode=&orderby=&page=2&rows=30'
    response = requests.post(url, headers=headers).text.split(',')
    res_page = '(?<="totalPage":).'
    responses = []
    for i in response:
        st_page = re.findall(res_page,i)
        st_page = ''.join(st_page)
        if len(st_page) != 0:
            for i in range(1,int(st_page)+1):
                # 获取多页信息
                url = f'http://www.cninfo.com.cn/data/statis/getMarketStatisticsData?sdate={date}&edate={date}&platecode=&orderby=&page={i}&rows=30'
                response = requests.post(url, headers=headers).text
                responses.append(response)
                # time.sleep(random.randint(3,6))
    return responses

def get_info(response):    
    # 分析数据并保存(response必须是text格式文件)
    # 创建信息列表
    code = []
    name = []
    time = []
    type = []
    buy_total = []
    sell_total = []
    buy_rate = []
    sell_rate = []

    # 正则表达式
    response1 = response.split(',')
    res_code = '(?<="secCode":").*(?=")'
    res_name = '(?<="secName":").*(?=")'
    res_time = '(?<="tradeTime":").*(?=")'
    res_type = '(?<="type":").*(?=")'
    res_buy_rate = '(?<="buyPercent":").*(?=")'
    res_sell_rate = '(?<="sellPercent":").*(?=")'
    response2 = response.split(',"')
    res_buy_total = '(?<=buyTotal":").*(?=")'
    res_sell_total = '(?<=sellTotal":").*(?=")'

    # 解析数据
    for i in response1:
        st_code = re.findall(res_code, i)
        st_name = re.findall(res_name, i)
        st_time = re.findall(res_time, i)
        st_type = re.findall(res_type, i)
        st_buy_rate = re.findall(res_buy_rate, i)
        st_sell_rate = re.findall(res_sell_rate, i)
        if len(st_code) != 0:
            code.append(''.join(st_code))
        if len(st_name) != 0:
            name.append(''.join(st_name))
        if len(st_time) != 0:
            time.append(''.join(st_time))
        if len(st_type) != 0:
            type.append(''.join(st_type))
        if len(st_buy_rate) != 0:
            buy_rate.append(''.join(st_buy_rate))
        if len(st_sell_rate) != 0:
            sell_rate.append(''.join(st_sell_rate))

    for i in response2:
        st_buy_total = re.findall(res_buy_total, i)
        st_sell_total = re.findall(res_sell_total, i)
        if len(st_buy_total) != 0:
            buy_total.append(''.join(st_buy_total))
        if len(st_sell_total) != 0:
            sell_total.append(''.join(st_sell_total))

    # 保存数据
    addr = f'data/stock{time[1]}.csv'
    with open(addr, 'a+') as f:
        for i in range(len(name)):
            f.write(str(code[i]))
            f.write(',')
            f.write(str(name[i]))
            f.write(',')
            f.write(str(time[i]))
            f.write(',')
            f.write(str(buy_rate[i]))
            f.write(',')
            f.write(str(sell_rate[i]))
            f.write(',')
            f.write(str(buy_total[i]))
            f.write(',')
            f.write(str(sell_total[i]))
            f.write(',')
            f.write(str(type[i]))
            f.write('\n')

In [120]:
responses = get_responses('2024-01-02')
for i in responses:
    get_info(i)